In [1]:
%pip install -q ollama
%pip install -q chromadb
%pip install -q pandas
%pip install -q numpy
%pip install -q tiktoken


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is avail

### Data Preparation

In [2]:
import os
import logging
import pandas as pd
import numpy as np
from dotenv import load_dotenv

# Load environment variables
try:
    load_dotenv()
    logging.info("Environment variables loaded successfully.")
except Exception as e:
    logging.error(f"Error loading environment variables: {e}")

In [3]:
# Configure logging
try:
    logging.basicConfig(
        filename="file_processing.log",
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s"
    )
    logging.info("Logging configured successfully.")
except Exception as e:
    print(f"Error configuring logging: {e}")
    raise

In [ ]:
import os
import logging

DATA_PATH = "./data"
CURRENT_FILE = "embedding_model_dataset_6.pkl"
try:
    if not os.path.exists(DATA_PATH):
        os.makedirs(DATA_PATH)
        logging.info(f"Created directory: {DATA_PATH}")
    else:
        logging.info(f"Directory already exists: {DATA_PATH}")
except Exception as e:
    logging.error(f"Error creating directory {DATA_PATH}: {e}")
    raise

In [5]:
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# List all .pkl files in the directory
try:
    pkl_files = [file for file in os.listdir(DATA_PATH) if file.endswith(".pkl")]
    logging.info(f"Found {len(pkl_files)} .pkl files in the directory.")
except Exception as e:
    logging.error(f"Error listing .pkl files in {DATA_PATH}: {e}")
    raise

if not pkl_files:
    logging.info("No .pkl files found in the directory.")
    print("No .pkl files found in the directory.")
    CURRENT_FILE = input("Enter the name of the file to save the DataFrame (default: pickle_part_9.pkl): ") or "pickle_part_9.pkl"
    logging.info(f"User entered file name: {CURRENT_FILE}")
else:
    print("Available .pkl files:")
    logging.info("Listing available .pkl files:")
    for idx, file in enumerate(pkl_files, start=1):
        print(f"{idx}. {file}")
        logging.info(f"{idx}. {file}")
    
    # Prompt the user to select a file
    choice = input(f"Select a file by number (or press Enter to use the default: pickle_part_9.pkl): ")
    if choice.isdigit() and 1 <= int(choice) <= len(pkl_files):
        CURRENT_FILE = pkl_files[int(choice) - 1]
        logging.info(f"User selected file: {CURRENT_FILE}")
    else:
        CURRENT_FILE = "pickle_part_9.pkl"
        logging.info(f"User did not select a valid file. Defaulting to: {CURRENT_FILE}")

logging.info(f"Final selected file for processing: {CURRENT_FILE}")
CURRENT_FILE_PATH = os.path.join(DATA_PATH, CURRENT_FILE)

Available .pkl files:
1. Embedding Model Dataset 2.pkl
2. Embedding Model Dataset 3.pkl
3. Embedding Model Dataset 1.pkl
4. Embedding Model Dataset 4.pkl
5. embedding_model_dataset.pkl
6. Embedding Model Dataset 5.pkl
7. Embedding Model Dataset 6.pkl


In [6]:
import logging
import pandas as pd

try:
    df = pd.read_pickle(CURRENT_FILE_PATH)
    logging.info(f"DataFrame loaded successfully from {CURRENT_FILE_PATH}.")
except Exception as e:
    logging.error(f"Error loading DataFrame from {CURRENT_FILE_PATH}: {e}")
    raise

df.head()

,id,title,artist,year,views,features,tag,language,cleaned_lyrics
0,2209308,Grinch Schedule Dialogue,Jim Carrey,2000,153264,{},pop,en,the nerve of those whos! inviting me done ther...
1,1625031,I Almost Let Go,Kurt Carr & The Kurt Carr Singers,2000,125007,{},pop,en,i almost let go i felt like i couldn't take li...
2,1359739,I’m the Map,Dora the Explorer,2000,115460,"{""Marc Weiner""}",pop,en,if there's a place you got to go i'm the one y...
3,1395900,Dora The Explorer Theme,Dora the Explorer,2000,109832,"{""Harrison Chad"",""Marc Weiner"",""Kathleen Herles""}",pop,en,"dora! boots! come on, dora do-do-do-do-dora do..."
4,1786575,I Hope You Dance,Lee Ann Womack,2000,80415,{},country,en,i hope you never lose your sense of wonder you...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15422 entries, 0 to 15421
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              15422 non-null  int64 
 1   title           15422 non-null  object
 2   artist          15422 non-null  object
 3   year            15422 non-null  int64 
 4   views           15422 non-null  int64 
 5   features        15422 non-null  object
 6   tag             15422 non-null  object
 7   language        15422 non-null  object
 8   cleaned_lyrics  15422 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.1+ MB


In [8]:
# Replace all NaN values in a dataframe with an empty string
df = df.fillna("")

# Rename cleaned_lyrics to lyrics for consistency
df.rename(columns={"cleaned_lyrics": "lyrics"}, inplace=True)

# Create three values for chroma embeddings
# Documents, IDs, and Metadatas
# Format the lyrics by removing new lines and extra spaces
df["lyrics"] = df["lyrics"].str.replace("\n", " ").str.replace(r"\s+", " ", regex=True).str.strip()

# Convert the lyrics column to a list
documents = df["lyrics"].tolist()

# Convert the ids to a string to keep them in a list
ids = df["id"].astype(str).tolist()

# Metadata includes all columns except "lyrics" and "id"
metadata = df.drop(columns=["lyrics", "id"]).to_dict(orient="records")

In [9]:
print("Number of documents:", len(documents))
print("Number of IDs:", len(ids))
print("Number of metadata entries:", len(metadata))

Number of documents: 15422
Number of IDs: 15422
Number of metadata entries: 15422


### Choma DB embed pushing

In [10]:
from chromadb.utils.embedding_functions.ollama_embedding_function import (OllamaEmbeddingFunction)

ollama_ef = OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings", # Corrected URL
    model_name="nomic-embed-text"
)

In [11]:
COLLECTION_NAME = os.getenv("COLLECTION_NAME") or "test"

In [ ]:
collection = client.get_or_create_collection(name=COLLECTION_NAME, embedding_function=ollama_ef)

In [13]:
%%time
from tqdm import tqdm

# Define batch size
BATCH_SIZE = 1000

# Split the data into batches
batches = [
    (documents[i:i + BATCH_SIZE], ids[i:i + BATCH_SIZE], metadata[i:i + BATCH_SIZE])
    for i in range(0, len(documents), BATCH_SIZE)
]

# Add documents to the collection in batches with progress tracking
print("Adding documents to the collection in batches...")
logging.info("Starting batch addition of documents to the collection.")

# Initialize tqdm progress bar
with tqdm(total=len(batches), desc="Processing Batches", unit="batch") as pbar:
    for i, (doc_batch, id_batch, meta_batch) in enumerate(batches):
        try:
            # Add the current batch to the collection
            collection.add(documents=doc_batch, ids=id_batch, metadatas=meta_batch)
            logging.info(f"Batch {i + 1}/{len(batches)} added successfully.")
        except Exception as e:
            # Log and print the error for the current batch
            logging.error(f"Error adding batch {i + 1}/{len(batches)}: {e}")
            print(f"Error adding batch {i + 1}/{len(batches)}: {e}")
        finally:
            # Update the progress bar regardless of success or failure
            pbar.update(1)

# Print the final count of documents in the collection
final_count = collection.count()
print(f"Number of documents in the collection '{COLLECTION_NAME}': {final_count}")
logging.info(f"Final document count in the collection '{COLLECTION_NAME}': {final_count}")

In [14]:
client.list_collections()

[Collection(name=lyric_embeddings)]

In [ ]:
import pandas as pd
import tiktoken

# Load your DataFrame
# df = pd.read_csv("your_file.csv") or define df here

# Choose a model encoding
encoding = tiktoken.encoding_for_model("text-embedding-3-small")  # or "gpt-3.5-turbo", etc.

# Count for documents, ids, and metadata
text = " ".join(documents)  # Concatenate all documents into a single string for token counting
text += " ".join(ids)  # Add ids to the text
text += " ".join([str(meta) for meta in metadata])  # Add metadata to the text
# Count tokens
tokens = encoding.encode(text)
print(f"Number of tokens: {len(tokens)}")

Number of tokens: 55652810
